# trotter

In [2]:
import numpy as np
import pytreenet as ptn
from copy import deepcopy
from scipy.linalg import expm

In [3]:
#               __________ 1 _________         
#              |           |          |              
#              2      _____6______    4  
#              |     |            |   |               
#              3     8            7   5            
node1, tensor1 = ptn.random_tensor_node((5, 6, 3, 2), identifier="site1")
node2, tensor2 = ptn.random_tensor_node((5, 4, 2), identifier="site2")
node3, tensor3 = ptn.random_tensor_node((4, 2), identifier="site3")
node4, tensor4 = ptn.random_tensor_node((6, 3, 2), identifier="site4")
node5, tensor5 = ptn.random_tensor_node((3, 2), identifier="site5")
node6, tensor6 = ptn.random_tensor_node((3, 5, 4, 2), identifier="site6")
node7, tensor7 = ptn.random_tensor_node((5, 2), identifier="site7")
node8, tensor8 = ptn.random_tensor_node((4, 2), identifier="site8")

ttn = ptn.TreeTensorNetworkState()

ttn.add_root(node1, tensor1)
ttn.add_child_to_parent(node2, tensor2, 0, "site1", 0)
ttn.add_child_to_parent(node3, tensor3, 0, "site2", 1)
ttn.add_child_to_parent(node4, tensor4, 0, "site1", 1)
ttn.add_child_to_parent(node5, tensor5, 0, "site4", 1)
ttn.add_child_to_parent(node6, tensor6, 0, "site1", 2)
ttn.add_child_to_parent(node7, tensor7, 0, "site6", 1)
ttn.add_child_to_parent(node8, tensor8, 0, "site6", 2)


In [4]:
orthogonality_center_id = "site1"
u_legs = tuple(range(ttn.nodes[orthogonality_center_id].nchildren()))
v_legs = (ttn.tensors[orthogonality_center_id].ndim-1,)
u_legs , v_legs

((0, 1, 2), (3,))

In [5]:
X, _, Z = ptn.pauli_matrices()

#      ttn.nearest_neighbours() = [('site1', 'site2'),
#                                  ('site1', 'site4'),
#                                  ('site1', 'site6'),
#                                  ('site2', 'site3'),
#                                  ('site4', 'site5'),
#                                  ('site6', 'site7'),
#                                  ('site6', 'site8')]
swaplist = ptn.SWAPlist(ttn.nearest_neighbours())

swap_op_list = swaplist.into_operators(ttn)
# operator_list[n].operator == ptn.swap_gate(2).reshape((2,2,2,2))
print(swap_op_list[0].operator.shape ,  swap_op_list[0].node_identifiers)
print(swap_op_list[1].operator.shape ,  swap_op_list[1].node_identifiers) 

(2, 2, 2, 2) ['site1', 'site2']
(2, 2, 2, 2) ['site1', 'site4']


In [6]:
tensor_products = []
for pairs in ttn.nearest_neighbours():
    term = ptn.TensorProduct({pairs[0]: Z, pairs[1]: Z})
    tensor_products.append(term)

# tensor_products = [ {"site1" : Z , "site2" : Z},
#                     {"site1" : Z , "site4" : Z},
#                     {"site1" : Z , "site6" : Z},
#                     {"site2" : Z , "site3" : Z},
#                     {"site4" : Z , "site5" : Z},
#                     {"site6" : Z , "site7" : Z},
#                     {"site6" : Z , "site8" : Z}]   

In [7]:
splitting_int = [3, 0, 1, 4, 2, 5, 6]
splitting_tuple = [(3, 1), (0, 1), (1, 1), (4, 1), (2, 1), (5, 1), (6, 1)]

swaps_before = [ptn.SWAPlist([]) for i in splitting_tuple]
swaps_before[0].extend([("site3", "site2"), ("site2", "site1")])

swaps_after = [ptn.SWAPlist([]) for i in range(len(splitting_tuple))]
swaps_after[0].extend([("site8", "site6"), ("site6", "site7")])

In [8]:
trotter = ptn.TrotterSplitting(tensor_products,
                     splitting_tuple,
                     swaps_before,
                     swaps_after)
unitary_operators = trotter.exponentiate_splitting(delta_time=0.1 , ttn=ttn)
len(unitary_operators) # 2 + 1 + 2 + 6

11

In [9]:
unitary_operators[1].node_identifiers 

['site2', 'site1']

In [10]:
unitary_operators[1].operator == ptn.swap_gate(2).reshape((2,2,2,2))  

array([[[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]],


       [[[ True,  True],
         [ True,  True]],

        [[ True,  True],
         [ True,  True]]]])

In [11]:
delta_time = 0.1
unitary_operators = [] 
i=0
split = (3,1)
tensor_product = tensor_products[split[0]]
# tensor_product = {"site2" : Z , "site3" : Z}
factor = -1j * split[1] * delta_time
exponentiated_operator = tensor_product.exp(factor)
# exponentiated_operator.operator = exp(factor*Z(site2)*Z(site3)) 
# exponentiated_operator.node_identifiers = ['site2', 'site3']

swaps_before = swaps_before[0].into_operators(ttn)
# swaps_before [ swap(1,2) with node_identifiers = ['site1', 'site2']
#               swap(2,3) with node_identifiers = ['site2', 'site3']]
swaps_after = swaps_after[0].into_operators(ttn)
# swaps_after [ swap(3,2) with node_identifiers = ['site3', 'site2']
#               swap(2,1) with node_identifiers = ['site2', 'site1']]

#            unitary_operators.extend(swaps_before)
#            unitary_operators.append(exponentiated_operator)
#            unitary_operators.extend(swaps_after)

In [12]:
tensor_products = []
for pairs in ttn.nearest_neighbours():
    term = ptn.TensorProduct({pairs[0]: Z, pairs[1]: Z})
    tensor_products.append(term)

# tensor_products = [ {"site1" : Z , "site2" : Z},
#                     {"site1" : Z , "site4" : Z},
#                     {"site1" : Z , "site6" : Z},
#                     {"site2" : Z , "site3" : Z},
#                     {"site4" : Z , "site5" : Z},
#                     {"site6" : Z , "site7" : Z},
#                     {"site6" : Z , "site8" : Z}]

splitting = [(3,2), 0, (1,4), 4, 2, 5, 6]

trotter = ptn.TrotterSplitting(tensor_products, splitting)
# if splitting = None ---> splitting = [0,1,2,3,4,5,6]
unitary_operators = trotter.exponentiate_splitting(delta_time=0.1 , ttn=ttn)
len(unitary_operators) == 7
unitary_operators[0].operator == scipy.linalg.expm(-1j * 2 * 0.1 * np.kron(Z, Z)).reshape((2,2,2,2))
unitary_operators[0].node_identifiers == ['site2', 'site3']
# order : kron(tensor_products["site2"], tensor_products["site2"]) 
unitary_operators[1].operator == scipy.linalg.expm(-1j * 1 * 0.1 * np.kron(Z, Z)).reshape((2,2,2,2))
unitary_operators[1].node_identifiers == ['site1', 'site2']


NameError: name 'scipy' is not defined

# TEBD


In [ ]:
# ._intital_state          \ .state (deep copy of initial state) 
# .trotter_splitting       \ ._num_time_steps
# ._time_step_size         \ 
# ._final_time             \ ._exponents = trotter_splitting.exponentiate_splitting 
# .operator (not implanted)                (time_step_size,state)
         
                                                       


In [ ]:
TEBD = ptn.TEBD(initial_state = ttn,
                trotter_splitting = trotter,
                time_step_size= 0.1,
                final_time= 10,
                operators =[ptn.random_tensor_product(ttn,1)])

In [ ]:
TEBD.state.tensors["site6"].shape , TEBD.state.tensors["site7"].shape 

((3, 5, 4, 2), (5, 2))

In [ ]:
TEBD.run()

100%|██████████| 101/101 [00:00<00:00, 143.61it/s]


In [ ]:
TEBD.state.tensors["site6"].shape , TEBD.state.tensors["site7"].shape 

((8, 2, 2, 2), (2, 2))

In [ ]:
TEBD.results

array([[ 3.90945789e+05 -633890.521626j  ,
         5.15031734e+05 -546425.63620398j,
         6.00892177e+05 -449511.49358909j,
         6.52349909e+05 -334385.3866706j ,
         6.90042452e+05 -188779.82099765j,
         7.42673396e+05   -3300.2891706j ,
         8.35316155e+05 +222890.63241016j,
         9.78922535e+05 +479164.25608288j,
         1.16477013e+06 +744412.34298755j,
         1.36585711e+06 +991693.30766453j,
         1.54482295e+06+1195023.75148757j,
         1.66568855e+06+1336047.29554919j,
         1.70536673e+06+1408347.09381241j,
         1.66096442e+06+1417979.25792451j,
         1.55035604e+06+1380108.95217163j,
         1.40584523e+06+1312974.03440492j,
         1.26313282e+06+1231314.36769898j,
         1.14943477e+06+1141570.09382716j,
         1.07487734e+06+1040512.40275589j,
         1.03013944e+06 +917750.0071579j ,
         9.91118659e+05 +761180.77951685j,
         9.28934907e+05 +563411.51424344j,
         8.21713817e+05 +326815.81346257j,
         6.

# tdvp_util


In [ ]:
#               __________ 1 _________         
#              |           |          |              
#              2           5          3      
#                                     |          
#                                     4  

ttn = ptn.TreeTensorNetwork()
node1, tensor1 = ptn.random_tensor_node((2, 2, 2, 2), identifier="id1")
node2, tensor2 = ptn.random_tensor_node((2, 2), identifier="id2")
node3, tensor3 = ptn.random_tensor_node((2, 2, 5), identifier="id3")
node4, tensor4 = ptn.random_tensor_node((2, 2), identifier="id4")
node5, tensor5 = ptn.random_tensor_node((2, 2), identifier="id5")
ttn.add_root(node1, tensor1)
ttn.add_child_to_parent(node2, tensor2, 0, "id1", 0)
ttn.add_child_to_parent(node3, tensor3, 0, "id1", 1)
ttn.add_child_to_parent(node4, tensor4, 0, "id3", 1)
ttn.add_child_to_parent(node5, tensor5, 0, "id1", 2)

tensor = ptn.crandn([2,2,5,2,2,
                     2,2,5,2,2])
leg_dict = {"id1": 0, "id2": 1, "id3": 2, "id4": 3, "id5": 4}
ttno = ptn.TTNO.from_tensor(ttn, tensor, leg_dict)

tp = ptn.TensorProduct({"id1": "1",
                        "id2": "2",
                        "id3": "3",
                        "id4": "4",
                        "id5": "5"})
conversion_dictionary = {"1": ptn.crandn((2, 2)),
                         "2": ptn.crandn((2, 2)),
                         "3": ptn.crandn((5, 5)),
                         "4": ptn.crandn((2, 2)),
                         "5": ptn.crandn((2, 2))}
H = ptn.Hamiltonian( tp , conversion_dictionary)
ttno = ptn.TTNO.from_hamiltonian( H, ttn)

In [ ]:
CST_id3 = ptn.CachedSiteTensor(ttn.nodes["id3"] , ttno.nodes["id3"] ,
                              ttn.tensors["id3"], ttno.tensors["id3"])
# tensor = CachedSiteTensor.compute()
CST_id3.compute().shape
# only works when all nodes have one open leg

AttributeError: module 'pytreenet' has no attribute 'CachedSiteTensor'

# PartialTreeCach

In [ ]:
"""
                |2
                |
                r
               / \\
         3|  5/  6\\   |4
          |  /     \\  |
           c1        c2
"""
state = ptn.random_small_ttns()
tensor_prod = [ptn.TensorProduct({"c1": "I3", "root": "root_op1", "c2": "I4"}),
               ptn.TensorProduct({"c1": "c1_op", "root": "root_op1", "c2": "I4"}),
               ptn.TensorProduct({"c1": "c1_op", "root": "root_op2", "c2": "c2_op"}),
               ptn.TensorProduct({"c1": "c1_op", "root": "I2", "c2": "c2_op"})]
conversion_dict = {"root_op1": ptn.random_hermitian_matrix(),
                   "root_op2": ptn.random_hermitian_matrix(),
                   "I2": np.eye(2),
                   "c1_op": ptn.random_hermitian_matrix(size=3),
                   "I3": np.eye(3),
                   "c2_op": ptn.random_hermitian_matrix(size=4),
                   "I4": np.eye(4)}
ham = ptn.Hamiltonian(tensor_prod, conversion_dict)
hamiltonian = ptn.TTNO.from_hamiltonian(ham, state)

In [ ]:
partial_tree_cache = ptn.PartialTreeCachDict()

# tdvp._initialize_partial_tree_cache():

# tdvp.update_tree_cache(node_ide,next_node_ide):
node_id = "c1"
next_node_id = "root"
tensor = ptn.contract_any(node_id, next_node_id,
                         state, hamiltonian,
                         partial_tree_cache)
partial_tree_cache.add_entry(node_id, next_node_id, tensor)

# tdvp.update_tree_cache(node_ide,next_node_ide):
node_id = "root"
next_node_id = "c2"
tensor = ptn.contract_any(node_id, next_node_id,
                         state, hamiltonian,
                         partial_tree_cache)
partial_tree_cache.add_entry(node_id, next_node_id, tensor)

np.allclose(partial_tree_cache.get_entry("root", "c2"), tensor)

True

# TDVPUpdatePathFinder

In [ ]:
ttn = ptn.random_big_ttns_two_root_children()
"""
                0
               / \\
              /   \\
             1     6
            / \\    \\
           /   \\    \\
          2     3     7
               / \\
              /   \\
             4     5
"""

'\n                0\n               / \\\n              /   \\\n             1     6\n            / \\    \\\n           /   \\    \\\n          2     3     7\n               / \\\n              /   \\\n             4     5\n'

In [ ]:
PF = ptn.TDVPUpdatePathFinder(ttn)
print( PF.start , PF.main_path)
print( PF.path_for_branch("site4"))
print( PF.path_for_branch("site3"))
print( PF.path_for_branch("site1"))
print( PF.path_for_branch("site0"))
print( PF.find_furthest_non_visited_leaf(['site4', 'site3', 'site1', 'site0']))
print( PF.find_furthest_non_visited_leaf(["site4","site5","site3","site2","site1"]))
print( PF.find_main_path_down_from_root(["site7","site6","site3"]))
print( PF.path_down_from_root(["site4","site5","site3","site2","site1"]))
print( PF.find_path())


site4 ['site4', 'site3', 'site1', 'site0']
['site4']
['site5', 'site3']
['site2', 'site1']
['site7', 'site6', 'site0']
site5
site7
['site0', 'site1', 'site3', 'site4']
['site0', 'site6', 'site7']
['site4', 'site5', 'site3', 'site2', 'site1', 'site0', 'site6', 'site7']


# TimeEvolution

In [ ]:
ttn = ptn.random_big_ttns_two_root_children()
tp1 = ptn.random_tensor_product(ttn, 3)
tp2 = ptn.random_tensor_product(ttn, 4)

TE = ptn.TTNTimeEvolution(initial_state = ttn,
                       time_step_size= 0.1,
                       final_time= 10,
                       operators = [tp1, tp2])
print(TE.num_time_steps)  
# TE.results --> Currently there are no results
print(TE.evaluate_operators())
print(TE._operator_index_dict)


100
[-3716.88236212-664.0096184j    462.31847607 +32.62464157j]
{}


In [ ]:
TE = ptn.TimeEvolution(initial_state = ttn,
                       time_step_size= 0.1,
                       final_time= 10,
                       operators = [tp1, tp2])
print(TE._operator_index_dict)


{}


In [ ]:
ttn = ptn.random_big_ttns_two_root_children()
tp = ptn.random_tensor_product(ttn, len(ttn.nodes))

psi, contraction_order = ttn.completely_contract_tree(to_copy=True)
op = tp.into_operator(order = contraction_order)
hamiltonian = op.operator

print( psi.shape , hamiltonian.shape)

# e^(-i*H*delta_time) |psi> (forward)
psi_time_evolved = ptn.time_evolve(psi = psi,
                                   hamiltonian = hamiltonian,
                                   time_difference = 0.1,
                                   forward = True)

(2, 2, 2, 2, 2, 2, 2, 2) (256, 256)


# TDVPAlgorithm

In [ ]:
# .initial_state , .hamiltonian , .time_step_size , .final_time , .operators
# TDVP.update_path == PF.find_path()

In [ ]:
"""
                0
               / \\
              /   \\
             1     6
            / \\    \\
           /   \\    \\
          2     3     7
               / \\
              /   \\
             4     5
"""
ref_tree = random_big_ttns_two_root_children2()
tp1 = ptn.TensorProduct({"site0": "0","site1": "1"})
tp2 = ptn.TensorProduct({"site2": "2","site3": "3","site4": "4"})
tp3 = ptn.TensorProduct({"site5": "5","site6": "6","site7": "7"})

conversion_dictionary = {"0": ptn.random_hermitian_matrix(2),
                         "1": ptn.random_hermitian_matrix(3),
                         "2": ptn.random_hermitian_matrix(4),
                         "3": ptn.random_hermitian_matrix(5),
                         "4": ptn.random_hermitian_matrix(6),
                         "5": ptn.random_hermitian_matrix(7),
                         "6": ptn.random_hermitian_matrix(8),
                         "7": ptn.random_hermitian_matrix(9),
                         "I2" : np.eye(2),"I3" : np.eye(3),"I4" : np.eye(4),"I5" : np.eye(5),
                         "I6" : np.eye(6),"I7" : np.eye(7),"I8" : np.eye(8),"I9" : np.eye(9)}
H = ptn.Hamiltonian( [tp1,tp2,tp3] , conversion_dictionary)
H = H.pad_with_identities(ref_tree)
hamiltonian = ptn.TTNO.from_hamiltonian(H, ref_tree) 

tdvp = ptn.TDVPAlgorithm(ref_tree, hamiltonian, 0.1 , 1,
                        ptn.TensorProduct({"site0": ptn.pauli_matrices()[0]}))                      

In [ ]:
# initial partial tree cache is besed on (rev_update_path , next_node_id_dict[rev_update_path])
rev_update_path , next_node_id_dict = tdvp._find_caching_path()
rev_update_path , next_node_id_dict 

(['site7', 'site6', 'site0', 'site2', 'site1', 'site5', 'site3', 'site4'],
 {'site0': 'site1',
  'site1': 'site3',
  'site3': 'site4',
  'site7': 'site6',
  'site6': 'site0',
  'site2': 'site1',
  'site5': 'site3'})

In [ ]:
## update_tree_cache ## 

# ("site7", "site6")  --> contract_leaf
ket_node, ket_tensor = ref_tree["site7"]
bra_tensor = ket_tensor.conj()
ham_node, ham_tensor = hamiltonian["site7"]
# ham_tensor = (2, 9, 9) , bra_tensor = (2, 9)
bra_ham = np.tensordot(ham_tensor, bra_tensor,
                       axes=(ham_node.nneighbours(),  # 1
                             ket_node.nneighbours())) # 1
bra_ham_ket = np.tensordot(ket_tensor, bra_ham,
                          axes=(ket_node.nneighbours(),  # 1 
                                ham_node.nneighbours())) # 1
# bra_ham_ket.shape = (2, 2 ,2)

# ("site6", "site0") ---> contract_subtrees_using_dictionary


In [ ]:
non_init_pairs = [("site4","site3"),("site3","site5"),("site3","site1"),
                          ("site1","site2"),("site1","site0"),("site0","site6"),
                          ("site6","site7"),("site1","site2")]
for pair in non_init_pairs:
    tdvp.update_tree_cache(pair[0],pair[1])

In [ ]:
## _contract_all_except_node ##
#            (out)
#    |n-1     |n      |0           
#    |        |       |          
#    |--------H-------|            
#    |        |       |             
#    |2n      |2n+1   |n+1   
#            (in)

################## "site7" #####################
target_node , tensor  = hamiltonian["site7"]  # (2, 9, 9)
neighbours = target_node.neighbouring_nodes() # ['site6']

# for neighbours
neighbour_id = 'site6'
cached_tensor = tdvp.partial_tree_cache.get_entry('site6','site7')
tensor = np.tensordot(tensor, cached_tensor,axes=((0,1)))
tensor = tensor.transpose(tdvp._find_tensor_leg_permutation("site7")) # (3, 0, 2, 1)
# tensor.shape = (2, 9, 2, 9)

#    |1      |0       
#    |       |     
#    H-------|         
#    |       |     
#    |3      |2

tensor = tdvp._get_effective_site_hamiltonian("site7")
# tensor.shape = (18,18)

################## "site6" ######################
target_node , tensor  = hamiltonian["site6"]  # (2, 8, 8)
neighbours = target_node.neighbouring_nodes() # ['site0','site7']

# for neighbours
neighbour_id = 'site0'
cached_tensor = tdvp.partial_tree_cache.get_entry('site0','site6')
tensor = np.tensordot(tensor, cached_tensor,axes=((0,1)))

neighbour_id = 'site7'
cached_tensor = tdvp.partial_tree_cache.get_entry('site7','site6')
tensor = np.tensordot(tensor, cached_tensor,axes=((0,1)))
tensor = tensor.transpose(tdvp._find_tensor_leg_permutation("site6")) # (3, 0, 2, 1)
# tensor.shape = (2, 2, 8, 2, 2, 8)

#    |1       |2      |0     
#    |        |       |     
#    |--------H-------|         
#    |        |       |     
#    |4       |5      |3

tensor = tdvp._get_effective_site_hamiltonian("site7")
# first tranpose(0,1,2,3,4,5) then reshape (2*8*2,2*8*2)


(18, 18)


In [ ]:
tdvp1 = deepcopy(tdvp)
# _update_link("site1","site3")
tdvp1._split_updated_site("site1","site3")
# q_legs = ptn.LegSpecification("site0",["site2"],[3])
# r_legs = ptn.LegSpecification(None,["site3"],[])
# tdvp1.state.split_node_qr("site1", q_legs, r_legs,
#                          q_identifier="site1",
#                          r_identifier= tdvp.create_link_id("site1", "site3"))
# site1 = Q
# link_site1_with_site3 = R 
# update_tree_cache(Q,"site3")
tdvp1._time_evolve_link_tensor("site1","site3") # include _get_effective_link_hamiltonian
ham = tdvp1._get_effective_link_hamiltonian("site1", "site3")
# e^(i*ham*t) |link_site1_with_site3>  
# contract R to site0
# orthogonality center = site0

In [ ]:
tdvp.state.move_orthogonalization_center("site4")
tdvp._move_orth_and_update_cache_for_path(["site4","site3","site1"])

#1    # move orthogonality center to site3
      # update partial_tree_cache (site4,site3)
#2    # move orthogonality center to site1
      # update partial_tree_cache (site3,site1)

# FirstOrderOneSiteTDVP

In [31]:
"""
                0
               / \\
              /   \\
             1     6
            / \\    \\
           /   \\    \\
          2     3     7
               / \\
              /   \\
             4     5
"""
ref_tree = ptn.random_big_ttns_two_root_children()
hamiltonian = ptn.TTNO.from_hamiltonian(ptn.random_hamiltonian_compatible(),ref_tree)

tdvp = ptn.TDVPAlgorithm(ref_tree, hamiltonian, 0.1 , 1,
                        ptn.TensorProduct({"site0": ptn.pauli_matrices()[0]}))



In [ ]:
tdvp.update_path , tdvp.orthogonalization_path

(['site4', 'site5', 'site3', 'site2', 'site1', 'site0', 'site6', 'site7'],
 [['site3', 'site5'],
  ['site3'],
  ['site1', 'site2'],
  ['site1'],
  ['site0'],
  ['site6'],
  ['site7']])

In [ ]:
def find_smallest_distance_neighbour(node , distance_dict: dict[str, int]) -> str:
    """
    Finds identifier of the neighbour of node with the minimal distance in
    distance dict, i.e. minimum distance to the orthogonality center.

    Parameters
    ----------
    node : Node
        Node for which to search.
    distance_dict : dict
        Dictionary with the distance of every node to the orthogonality center.

    Returns
    -------
    minimum_distance_neighbour_id : str
        Identifier of the neighbour of node with minimum distance to in
        distance_dict.

    """
    neighbour_ids = node.neighbouring_nodes()
    neighbour_distance_dict = {neighbour_id: distance_dict[neighbour_id]
                               for neighbour_id in neighbour_ids}
    minimum_distance_neighbour_id = min(neighbour_distance_dict,
                                        key=neighbour_distance_dict.get)
    return minimum_distance_neighbour_id

In [ ]:
FO1s = ptn.FirstOrderOneSiteTDVP(ref_tree, hamiltonian, 0.1 , 1,
                                 [ptn.TensorProduct({"site0": ptn.pauli_matrices()[0]}),
                                  ptn.TensorProduct({"site0": ptn.pauli_matrices()[1]})])

In [ ]:
# inital orthogonality_center_id is update_path[0]
FO1s.update_path

['site4', 'site5', 'site3', 'site2', 'site1', 'site0', 'site6', 'site7']

In [ ]:
FO1s.orthogonalization_path

[['site3', 'site5'],
 ['site3'],
 ['site1', 'site2'],
 ['site1'],
 ['site0'],
 ['site6'],
 ['site7']]

In [ ]:
# run_one_time : 

# FO1s._first_update("site4")
# FO1s._normal_update("site5",1)
# FO1s._final_update("site3",2)
# ....
# FO1s._final_update("site7")

In [ ]:
# _first_update("site4")

# update_site("site4")

# next_node_id = orthogonalization_path[0][0] = "site3"
# update_link("site4","site3")

In [ ]:
# normal_update("site5",i=1)

# 
# _move_orth_and_update_cache_for_path(orthogonalization_path[i-1]= ['site3', 'site5'] )
#  update_site("site5")

# next_node_id = orthogonalization_path[i][0] = "site3"
# update_link("site5","site3")

In [ ]:
# final_update("site7")

# _move_orth_and_update_cache_for_path(['site7'])
# update_site("site7")

# move_orthogonalization_center("site4")
# self._init_partial_tree_cache()

# SecondOrderOneSiteTDVP

In [ ]:
SO1s = ptn.SecondOrderOneSiteTDVP(ref_tree, hamiltonian, 0.1 , 1,
                                 ptn.TensorProduct({"site0": ptn.pauli_matrices()[0]}))

In [ ]:
# SO1s.run_one_time()

# forward_sweep()
# final_forward_update()
# backward_sweep()

AttributeError: 'SecondOrderOneSiteTDVP' object has no attribute 'run_one_time'

In [ ]:
# forward_sweep() :
# update with 0.5 factor according to update_path until the last one 
# last opertation : _update_link(site6, site7)

# final_forward_update() :
# update last update_path[-1]="site7" with factor 1 
# _update_site(site7)

In [ ]:
SO1s.backwards_update_path

['site7', 'site6', 'site0', 'site1', 'site2', 'site3', 'site5', 'site4']

In [ ]:
SO1s.backwards_orth_path

[['site6'],
 ['site0'],
 ['site1'],
 ['site2', 'site1'],
 ['site3'],
 ['site5', 'site3'],
 ['site4']]

In [ ]:
# backward_sweep() :

#1 update_first_backward_link() : 
#     update_link(site7, backwards_update_path[1] = site6 , 0.5)
#2 normal_backward_update('site6', 2) : 
#     update_site(site6 , 0.5)
#     move_orth_and_update_cache_for_path( backwards_orth_path[2-1] )
#     update_link(site6, backwards_update_path[2+1]=site0 , 0.5)
# normal_backward_update('site0', 3)  
#      ....
# normal_backward_update('site5', 7) 

# final_backward_update('site4') :
#      update_site(site4 , 0.5)

In [ ]:
FO1s = ptn.FirstOrderOneSiteTDVP(ref_tree, hamiltonian, 0.1 , 1,
                                 [ptn.random_tensor_product(ref_tree, 3),
                                  ptn.TensorProduct({"site0": ptn.pauli_matrices()[1]})])

In [ ]:
# <state|tp|state>
tp = FO1s.operators[0]
FO1s.evaluate_operator(tp)

(-1320.5690191446304-744.783921936321j)

In [ ]:
# <state|FO1s.operators|state>
FO1s.evaluate_operators()

In [ ]:
FO1s._init_results(evaluation_time =2) 
# oprerator 1 exp. value   [_ ,_ ,_ ,_ ,_ ,_]     num_time_steps//evaluation_time + 1 elements
# oprerator 2 exp. value   [_ ,_ ,_ ,_ ,_ ,_]
# evaluation time          [0 ,.2, .4, .6, ,.8, 1]
FO1s.results

array([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [ ]:
FO1s._init_results(evaluation_time="inf")
# only evaluates at the final time
FO1s.results

array([[0.+0.j],
       [0.+0.j],
       [0.+0.j]])

In [ ]:
FO1s.run(evaluation_time = 2 )
print(FO1s.results)
# FO1s.operator_results(realise = True) = np.real(self.results[0:-1])
# FO1s.times() = FO1s.results[-1]
FO1s.results_real()

100%|██████████| 11/11 [00:00<00:00, 72.92it/s]

[[nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj]
 [nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj nan+nanj]
 [0.  +0.j 0.2 +0.j 0.4 +0.j 0.6 +0.j 0.8 +0.j 1.  +0.j]]


False

In [ ]:
SO1s = ptn.SecondOrderOneSiteTDVP(ref_tree, hamiltonian, 0.1 , 1,
                                 ptn.TensorProduct({"site0": ptn.pauli_matrices()[0]}))

SO1s.run(evaluation_time = 2 )
SO1s.results

100%|██████████| 11/11 [00:00<00:00, 36.33it/s]


array([[7.90500409e+03+9.09494702e-13j, 4.67107746e+03-9.09494702e-13j,
        5.16942669e+03-2.72848411e-12j, 9.08538912e+03+4.54747351e-13j,
        1.32292649e+04+4.54747351e-13j, 1.55440002e+04+2.27373675e-12j],
       [0.00000000e+00+0.00000000e+00j, 2.00000000e-01+0.00000000e+00j,
        4.00000000e-01+0.00000000e+00j, 6.00000000e-01+0.00000000e+00j,
        8.00000000e-01+0.00000000e+00j, 1.00000000e+00+0.00000000e+00j]])

# test_tdvp_on_mps

In [ ]:
_, tensor0 = ptn.random_tensor_node((4,2), "site_0")
_, tensor1 = ptn.random_tensor_node((4,5,3), "site_1")
_, tensor2 = ptn.random_tensor_node((5,6,4), "site_2")
_, tensor3 = ptn.random_tensor_node((6,5), "site_3")
tensor_list = [tensor0, tensor1, tensor2, tensor3]
mps = ptn.MatrixProductState.from_tensor_list(tensor_list,root_site=1,
                                             node_prefix="site_")
ref_mps = deepcopy(mps)

matrix = ptn.random_hermitian_matrix((2*3*4*5))
matrix = matrix.reshape(2,3,4,5,2,3,4,5)
leg_dict = {"site_"+str(i): i for i in range(4)}
mpo = ptn.TTNO.from_tensor(mps, matrix, leg_dict)


operators = [ptn.bosonic_operators(i)[2] for i in range(2,6)]
operators = [ptn.TensorProduct({f"site_{i}": op}) for i, op in enumerate(operators)]
operators.append(ptn.TensorProduct({f"site_{i-2}": np.eye(i) for i in range(2,6)}))

In [ ]:
tdvp = ptn.FirstOrderOneSiteTDVP(ref_mps, mpo, 0.1 , 1,operators)
tdvp.update_path , tdvp.orthogonalization_path              

(['site_3', 'site_2', 'site_1', 'site_0'],
 [['site_2'], ['site_1'], ['site_0']])

In [ ]:
from pytreenet.contractions.state_operator_contraction import (contract_leaf, 
                                                               contract_subtrees_using_dictionary)

ref_mps = deepcopy(tdvp.state) # tdvp.state is in canonical form


# initial partial tree cache
ref_cache_dict = ptn.PartialTreeCachDict()
cache0 = contract_leaf("site_0", ref_mps, mpo)
ref_cache_dict.add_entry("site_0", "site_1", cache0)
cache1 = contract_subtrees_using_dictionary("site_1", "site_2",
                                                    ref_mps, mpo,
                                                    ref_cache_dict)
ref_cache_dict.add_entry("site_1", "site_2", cache1)
cache_2 = contract_subtrees_using_dictionary("site_2", "site_3",
                                                     ref_mps, mpo,
                                                     ref_cache_dict)
ref_cache_dict.add_entry("site_2", "site_3", cache_2)
# ref_cache_dict = tdvp.partial_tree_cache

# tdvp._first_update("site_3") : 
# tdvp.update_site("site_3") :
hamiltonian_eff_site = tdvp._get_effective_site_hamiltonian("site_3") # (30,30)
hamiltonian_eff_site2 = np.tensordot(mpo.tensors["site_3"],
                                     tdvp.partial_tree_cache.get_entry("site_2", "site_3"),
                                     axes=(mpo.nodes["site_3"].neighbour_index("site_2"),1))
hamiltonian_eff_site2 = np.transpose(hamiltonian_eff_site2, (3,0,2,1)).reshape(30,30)
print(np.allclose(hamiltonian_eff_site2, hamiltonian_eff_site))

psi = tdvp.state.tensors["site_3"]
tdvp.state.tensors["site_3"] = ptn.time_evolve(psi,hamiltonian_eff_site,
                                                   tdvp.time_step_size,
                                                   forward=True)

u = expm(-1j*0.1*hamiltonian_eff_site2)
u = u.reshape(6,5,6,5)
ref_mps.tensors["site_3"] = np.tensordot(u, ref_mps.tensors["site_3"],
                                                 axes=((2,3),(0,1)))

print(np.allclose(tdvp.state.tensors["site_3"] , ref_mps.tensors["site_3"]))

# tdvp.update_link("site_3","site_2") :
tdvp._split_updated_site("site_3","site_2")

q, r = ptn.tensor_qr_decomposition(ref_mps.tensors["site_3"],
                                    (len(ref_mps.nodes["site_3"].shape)-1, ), # 1
                                    (ref_mps.nodes["site_3"].neighbour_index("site_2"), ), # 0
                                    mode= ptn.SplitMode.KEEP)
ref_mps.tensors["site_3"] = q.transpose(1,0)
new_cache = contract_leaf("site_3", ref_mps, mpo)
ref_cache_dict.add_entry("site_3", "site_2", new_cache)

print(np.allclose(tdvp.state.tensors["site_3"] , ref_mps.tensors["site_3"]))
print(np.allclose(new_cache, tdvp.partial_tree_cache.get_entry("site_3", "site_2")))

tdvp._time_evolve_link_tensor("site_3","site_2")

heff = np.tensordot(ref_cache_dict.get_entry("site_2", "site_3"),
                    ref_cache_dict.get_entry("site_3", "site_2"),
                    axes=(1,1))
heff = np.transpose(heff, (1,3,0,2)).reshape(36,36)

u = expm(1j*0.1*heff)
u = u.reshape(6,6,6,6)
updated_r = np.tensordot(u,r,axes=((2,3),(1,0)))

np.allclose(heff, tdvp._get_effective_link_hamiltonian("site_3", "site_2"))

tdvp.state.contract_nodes('link_site_3_with_site_2', "site_2", new_identifier="site_2")

next_site = np.tensordot(ref_mps.tensors["site_2"],
                                 updated_r,
                                 axes=(1,0))
next_site = next_site.transpose(0,2,1)
ref_mps.tensors["site_2"] = next_site
ref_mps.orthogonality_center_id = "site_2"

print(np.allclose(tdvp.state.tensors["site_2"], ref_mps.tensors["site_2"]))


True
True
True
True
True
